In [1]:
import pandas as pd
import chess
import chess.pgn
import chess.engine
import io


In [2]:
eng = chess.engine.SimpleEngine.popen_uci("../../stockfish/stockfish")

In [3]:
df = pd.read_csv('../data/df_plays.csv')
df.head(3)

,id,Plays
0,0,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...
1,1168,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...
2,1772,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...


In [4]:
def get_move_index(n, turn='w'):
    if turn.lower() == 'b':
        return 2 * n - 1
    else:
        return 2 * n - 1 - 1

In [5]:
def get_family(row, move_number):
    # fen = 'r1bq1rk1/2p1bppp/p1n2n2/1p1pp3/4P3/1BP2N2/PP1P1PPP/RNBQR1K1 w - - 0 9'
    # board = chess.Board(fen=fen)
    # white_move = 2 * move_number - 1 - 1
    # black_move = 2 * move_number - 1

    pgn = io.StringIO(row['Plays'])

    game = chess.pgn.read_game(pgn)

    mainline = [m.san() for m in game.mainline()]

    row['SubVariation'] = '{0}. {1}'.format(
        move_number,
        mainline[get_move_index(move_number)]
    ) 

    row['Mainline'] = mainline
    
    return row

df = df.apply(lambda row: get_family(row, 9), axis=1)

,id,Plays,SubVariation,Mainline
0,0,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,9. exd5,"[e4, e5, Nf3, Nc6, Bb5, a6, Ba4, Nf6, O-O, Be7..."
1,1168,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,9. exd5,"[e4, e5, Nf3, Nc6, Bb5, a6, Ba4, Nf6, O-O, Be7..."
2,1772,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,9. exd5,"[e4, e5, Nf3, Nc6, Bb5, a6, Ba4, Nf6, O-O, Be7..."
3,2129,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,9. exd5,"[e4, e5, Nf3, Nc6, Bb5, a6, Ba4, Nf6, O-O, Be7..."
4,2522,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,9. exd5,"[e4, e5, Nf3, Nc6, Bb5, a6, Ba4, Nf6, O-O, Be7..."
...,...,...,...,...
2979,2113381,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,9. exd5,"[e4, e5, Nf3, Nc6, Bb5, a6, Ba4, Nf6, O-O, Be7..."
2980,2117758,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,9. exd5,"[e4, e5, Nf3, Nc6, Bb5, a6, Ba4, Nf6, O-O, Be7..."
2981,2118961,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,9. exd5,"[e4, e5, Nf3, Nc6, Bb5, a6, Ba4, Nf6, O-O, Be7..."
2982,2119588,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,9. exd5,"[e4, e5, Nf3, Nc6, Bb5, a6, Ba4, Nf6, O-O, Be7..."
